In [1]:
!pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 1.5 MB/s eta 0:00:0000:0100:01


In [28]:
from z3 import *

pessoas = ["Ana","Nuno","Pedro","Maria"]
gabs = [1,2,3]
x = {}
for p in pessoas:
    x[p] = {}
    for g in gabs:
        x[p][g] = Bool("%s,%d" % (p,g))

s = Solver()



# Cada pessoa ocupa pelo menos um gabinete.
for p in pessoas:
    s.add(Or([x[p][g] for g in gabs]))

# Cada pessoa ocupa no máximo um gabinete.
for p in pessoas:
    s.add(Sum([If(x[p][g], 1, 0) for g in gabs]) == 1)

# O Nuno e o Pedro não podem partilhar gabinete.
for g in gabs:
    s.add(Or(Not(x["Pedro"][g]), Not(x["Nuno"][g])))

# Se a Ana ficar sozinha num gabinete, então o Pedro também terá
# que ficar sozinho num gabinete.
#(falta)

# Cada gabinete só pode acomodar, no máximo, 2 pessoas.
for g in gabs:
    s.add(sum(If(x[p][g], 1, 0) for p in pessoas) == 2) #acho que está certo

s.push()

if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
else:
  print("Não tem solução.")

Ana fica no gabinete 2
Nuno fica no gabinete 2
Pedro fica no gabinete 1
Maria fica no gabinete 2
